In [ ]:
# ==== INSTALLAZIONE (solo se non hai già i pacchetti) ====
!pip install -q ibm_watson ibm_cloud_sdk_core ibm_watson_machine_learning gradio pandas openpyxl

# ==== IMPORTS ====
from ibm_watson import DiscoveryV2
from ibm_watson.discovery_v2 import QueryLargePassages
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_watson_machine_learning.foundation_models import Model
import json
import gradio as gr
import pandas as pd
from datetime import datetime
import os

# ==== CONFIGURAZIONE ====
discovery_project_id = "c1aeb15b-e664-4004-b93b-a3947513c9a1"
watsonx_project_id = "faa54735-bb10-485d-b252-7464d22c729f"
api_key = "u4qcYmojPps_P5KxKQNhpdO5KVqsPTUu-boJfljo5qFl"  # Discovery
cloud_account_api_key = 'XE5qCeWjlNNKZwzeB_wQ0JPFo_Yih9cudgk1JaklrdE7'  # Watsonx

service_url = "https://api.eu-de.discovery.watson.cloud.ibm.com/instances/ddfd011b-142f-4ea4-8836-f503c73969bf"
endpoint = "https://eu-de.ml.cloud.ibm.com"

# ==== AUTENTICAZIONE ====
authenticator = IAMAuthenticator(api_key)
discovery = DiscoveryV2(version='2020-08-30', authenticator=authenticator)
discovery.set_service_url(service_url)

# Access token per Watsonx
try:
    access_token = IAMTokenManager(
        apikey=cloud_account_api_key,
        url="https://iam.cloud.ibm.com/identity/token"
    ).get_token()
except Exception as e:
    print("Errore autenticazione:", e)

# ==== MODELLO Watsonx.ai ====
credentials = {
    "url": endpoint,
    "token": access_token
}

gen_params = {
    "DECODING_METHOD": "sample",
    "MAX_NEW_TOKENS": 500,
    "MIN_NEW_TOKENS": 2,
    "STREAM": False,
    "TEMPERATURE": 0.3,
    "TOP_K": 50,
    "TOP_P": 1,
    "RANDOM_SEED": 10
}

model_id = "ibm/granite-3-8b-instruct"
model = Model(model_id, credentials, gen_params, watsonx_project_id)

# ==== TEMPLATE PROMPT ====
prompt_template = """
Documento:
###
%s
###

Rispondi alla seguente domanda utilizzando le informazioni presenti in tutti i documenti.
Rispondi con frasi complete, utilizzando lettere maiuscole dove necessario e la punteggiatura corretta.
Cerca attentamente nei file la risposta.
Non inserire nella risposta altre domande o spiegazioni successive dopo aver risposto alla prima domanda.
Se non trovi la risposta corretta nel documento, rispondi "Non lo so".

Domanda: %s
Risposta: 
"""

# ==== FUNZIONI BASE ====
combined_disc_results = []

def query_discovery(question):
    passages = {
        "enabled": True,
        "per_document": True,
        "find_answers": True,
        "max_per_document": 1,
        "characters": 500
    }

    query_large_passages_model = QueryLargePassages.from_dict(passages)

    return discovery.query(
        project_id=discovery_project_id,
        natural_language_query=question,
        passages=query_large_passages_model,
        count=1
    ).get_result()

def augment(template_in, context_in, query_in):
    return template_in % (context_in, query_in)

def generate(model_in, augmented_prompt_in):
    generated_response = model_in.generate(augmented_prompt_in)

    if ("results" in generated_response
        and len(generated_response["results"]) > 0
        and "generated_text" in generated_response["results"][0]):
        return generated_response["results"][0]["generated_text"]
    else:
        return "Errore: risposta non generata correttamente."

# ==== SENTIMENT MANUALE ====
# Definizione di parole positive e negative
positive_words = ["buono","buona", "bene", "fantastico", "fantasticamente" "eccellente", "ottimo", "positivo", "positiva" "bellissimo", "bellissima", "ottima", "stupendo", "stupenda"]
negative_words = ["negativo", "pessimo", "pessima" "terribile", "brutta", "brutto", "malissimo", "scadente", "deludente", "delusione", "orribile"]

# Funzione per analizzare il sentiment manualmente
def manual_sentiment(feedback):
    feedback = feedback.lower()
    positive_count = sum(1 for word in positive_words if word in feedback)
    negative_count = sum(1 for word in negative_words if word in feedback)

    if positive_count > negative_count:
        return "POSITIVE"
    elif negative_count > positive_count:
        return "NEGATIVE"
    else:
        return "NEUTRAL"

# Funzione per processare il feedback e gestire il sentiment manuale
def process_feedback(feedback, question, risposta):
    try:
        # Analisi del sentiment manuale
        sentiment = manual_sentiment(feedback)

        # 3. Salvataggio su Excel
        data = {
            "Timestamp": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
            "Feedback": [feedback],
            "Sentiment": [sentiment]
        }

        df_new = pd.DataFrame(data)
        file_path = "feedback_watson.xlsx"

        if os.path.exists(file_path):
            df_existing = pd.read_excel(file_path)
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        else:
            df_combined = df_new

        df_combined.to_excel(file_path, index=False)

        return f"Feedback ricevuto e salvato con successo. Sentiment: {sentiment}", file_path

    except Exception as e:
        return f"Errore durante l'elaborazione del feedback: {str(e)}", None


# ==== FUNZIONE PRINCIPALE PER LA RICEZIONE DELLA DOMANDA E DEL FEEDBACK ====
def chat_watson_fase1(question):
    try:
        # 1. Risposta Watson
        discovery_json = query_discovery(question)
        combined_disc_results.clear()

        for doc_index in range(len(discovery_json["results"])):
            for j in range(len(discovery_json["results"][doc_index])):
                passages = discovery_json["results"][doc_index]["document_passages"]
                disc_results = []
                for item in passages:
                    item = item["passage_text"].replace("<em>", "").replace("</em>", "")
                    disc_results.append(item)
                combined_disc_results.append("\n".join(disc_results))

        augmented_prompt = augment(prompt_template, combined_disc_results, question)
        risposta = generate(model, augmented_prompt)

        return risposta

    except Exception as e:
        return f"Errore durante l'elaborazione della domanda: {str(e)}"


# ==== INTERFACCIA GRADIO DIVISA IN DUE FASI ====
def fase1_ui(question):
    risposta = chat_watson_fase1(question)
    return risposta

def fase2_ui(feedback, question, risposta):
    return process_feedback(feedback, question, risposta)


# Gradio per la **fase 1** (domanda e risposta)
iface1 = gr.Interface(
    fn=fase1_ui,
    inputs=gr.Textbox(lines=3, label="Fai una domanda"),
    outputs=gr.Textbox(lines=10, label="Risposta"),
    title="Fase 1 - Domanda e Risposta",
    description="Fai una domanda e ricevi una risposta."
)

# Gradio per la **fase 2** (feedback)
iface2 = gr.Interface(
    fn=fase2_ui,
    inputs=gr.Textbox(lines=2, label="Lascia un feedback (opzionale)"),
    outputs=[
        gr.Textbox(lines=5, label="Risultato Feedback"),
        gr.File(label="Scarica il file Excel con i feedback")
    ],
    title="Fase 2 - Feedback e Sentiment",
    description="Lascia un feedback sulla risposta e vedi il sentiment."
)

# Lancia Fase 1
iface1.launch(share=True)  # Lancia la fase 1

# Una volta che l'utente ha visto la risposta, lancia Fase 2
iface2.launch(share=True)  # Lancia la fase 2
